In [1]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_csv(GLOBAL.CLEANED_DATA_2014_2019)

In [4]:
df = df.iloc[:, 1:]

In [5]:
df = df.groupby(['player_id', 'tm', 'player', 'pos', 'season'], as_index=False)\
    .agg({
        'offensive_snapcount': np.sum,
        'offensive_snapcount_percentage': np.mean,
        'passing_rating': np.mean,
        'passing_yds': np.sum,
        'passing_td': np.sum,
        'passing_att': np.sum,
        'receiving_yds': np.sum,
        'receiving_td': np.sum,
        'receiving_rec': np.sum,
        'receiving_tar': np.sum,
        'rushing_att': np.sum,
        'standard_fantasy_points': np.sum,
        'ppr_fantasy_points': np.sum,
        'half_ppr_fantasy_points': np.sum
})

In [6]:
df = df.loc[df['season'] >= 2012]

In [7]:
#Change based on scoring format: "half_ppr", "ppr", or "standard"
scoring_format = "half_ppr"

pd.set_option('chained_assignment', None)

lag_features = [
    'rushing_att', 
    'receiving_tar', 
    'offensive_snapcount', 
    'offensive_snapcount_percentage',
    f'{scoring_format}_fantasy_points',
    'passing_rating',
    'passing_att', 
    'passing_td'
]

for lag in range(1, 6):
    shifted = df.groupby('player_id').shift(lag)
    for column in lag_features:
        df[f'lag_{column}_{lag}'] = shifted[column]

df = df.fillna(-1)